In [1]:
import sys
sys.path.append("/uscms/home/sbrightt/nobackup/iDM/iDMe_analysis/CMSSW_10_6_26/src/iDMeAnalysis/python_analysis/analysisTools")
from analysisImports import *
setLibParams()

cfgDir = "/uscms/home/sbrightt/nobackup/iDM/iDMe_analysis/CMSSW_10_6_26/src/iDMeAnalysis/python_analysis/configs/"
plt.style.use(cfgDir+'plt_settings.mplstyle')

In [2]:
import time
importlib.reload(aTools)
importlib.reload(subroutines)

start = time.time()
file = "/uscms_data/d3/sbrightt/iDM/iDMe_analysis/CMSSW_10_6_26/src/iDMeAnalysis/AODSkimmer/full_m42_ct1.root"
az = aTools.Analyzer(cfgDir+"sample_configs/signal_2018_aEM.json",cfgDir+'histo_configs/histoList.json')
histos = az.process()
end = time.time()
print("Took {0} minutes {1} seconds".format((end-start)//60, (end-start)%60))

Preprocessing:   0%|          | 0/340 [00:00<?, ?file/s]

Processing:   0%|          | 0/340 [00:00<?, ?chunk/s]

Took 23.0 minutes 10.62148118019104 seconds


In [7]:
coffea.util.save(histos,"histos.coffea")

MemoryError: 

In [5]:
reco_vtx_genMatch = histos['reco_vertex_genMatch']

for ax in reco_vtx_genMatch.axis('sample').identifiers():
    samp = ax.name
    outdir = 'plots/'+samp+'/recoVertex_genMatch/'
    os.makedirs(outdir,exist_ok=True)
    
    # plotting distance from gen to reconstructed vertex
    plt.figure(figsize=(10,8))
    h_dist = reco_vtx_genMatch.integrate('sample',int_range=samp).sum('chi2','dR',overflow='all')
    plt.yscale('log')
    hist.plot1d(h_dist,overlay="type")
    plt.xlim([0,50])
    plt.xticks(np.arange(0,51,5))
    plt.savefig(outdir+"distGenVtx_nearestToGen_byVtxType.pdf")
    plt.close()
    
    # plotting vertex reduced chi2
    plt.figure(figsize=(10,8))
    h_chi2 = reco_vtx_genMatch.integrate('sample',int_range=samp).sum('dist','dR',overflow='all')
    plt.yscale('log')
    hist.plot1d(h_chi2,overlay="type")
    plt.xlim([0,20])
    plt.xticks(np.arange(0,21,2))
    plt.savefig(outdir+"chi2_nearestToGen_byVtxType.pdf")
    plt.close()
    
    # plotting electron dR
    plt.figure(figsize=(10,8))
    h_dR = reco_vtx_genMatch.integrate('sample',int_range=samp).sum('dist','chi2',overflow='all').rebin("dR",2)
    #plt.yscale('log')
    hist.plot1d(h_dR,overlay="type")
    plt.savefig(outdir+"eeDR_nearestToGen_byVtxType.pdf")
    plt.close()

/uscms_data/d3/sbrightt/miniconda3/envs/coffea/lib/python3.8/site-packages/coffea/hist/plot.py:333: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  ax.set_ylim(0, None)
/uscms_data/d3/sbrightt/miniconda3/envs/coffea/lib/python3.8/site-packages/coffea/hist/plot.py:333: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  ax.set_ylim(0, None)
/uscms_data/d3/sbrightt/miniconda3/envs/coffea/lib/python3.8/site-packages/coffea/hist/plot.py:333: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  ax.set_ylim(0, None)
/uscms_data/d3/sbrightt/miniconda3/envs/coffea/lib/python3.8/site-packages/coffea/hist/plot.py:333: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  ax.set_ylim(0, None)
/uscms_data/d3/sbrightt/miniconda3/envs/coffea/lib/python3.8/site-packages/c

In [3]:
file = "/uscms_data/d3/sbrightt/iDM/iDMe_analysis/CMSSW_10_6_26/src/iDMeAnalysis/AODSkimmer/full_m42_ct1.root"
events = NanoEventsFactory.from_root(file,treepath='ntuples/outT',schemaclass=NanoAODSchema).events()

In [13]:
vtx_regreg = events.EleVertexRegReg
vtx_lowlow = events.EleVertexLowLow
vtx_lowreg = events.EleVertexLowReg
gen_ele = ak.flatten(events.GenPart[events.GenPart.ID == 11])

vtx_regreg_r = ak.zip(
    {
        "x": vtx_regreg.x,
        "y": vtx_regreg.y,
        "z": vtx_regreg.z
    },
    with_name="ThreeVector"
)

vtx_lowreg_r = ak.zip(
    {
        "x": vtx_lowreg.x,
        "y": vtx_lowreg.y,
        "z": vtx_lowreg.z
    },
    with_name="ThreeVector"
)

vtx_lowlow_r = ak.zip(
    {
        "x": vtx_lowlow.x,
        "y": vtx_lowlow.y,
        "z": vtx_lowlow.z
    },
    with_name="ThreeVector"
)

gen_vtx_r = ak.zip(
    {
        "x": gen_ele.x,
        "y": gen_ele.y,
        "z": gen_ele.z
    },
    with_name="ThreeVector"
)

rr_nearest = ak.argmin((vtx_regreg_r - gen_vtx_r).rho,axis=1,keepdims=True)
lr_nearest = ak.argmin((vtx_lowreg_r - gen_vtx_r).rho,axis=1,keepdims=True)
ll_nearest = ak.argmin((vtx_lowlow_r - gen_vtx_r).rho,axis=1,keepdims=True)

vtx_regreg_nearest = ak.flatten(vtx_regreg[rr_nearest])
vtx_lowreg_nearest = ak.flatten(vtx_lowreg[lr_nearest])
vtx_lowlow_nearest = ak.flatten(vtx_lowlow[ll_nearest])
all_vtx_nearest = ak.concatenate([vtx_regreg[rr_nearest],vtx_lowreg[lr_nearest],vtx_lowlow[ll_nearest]],axis=1)

dist_rr_nearest = (vtx_regreg_r[rr_nearest] - gen_vtx_r).rho
dist_lr_nearest = (vtx_lowreg_r[lr_nearest] - gen_vtx_r).rho
dist_ll_nearest = (vtx_lowlow_r[ll_nearest] - gen_vtx_r).rho

all_dist_nearest = ak.concatenate([dist_rr_nearest,dist_lr_nearest,dist_ll_nearest],axis=1)
nearest_ind = ak.argmin(all_dist_nearest,axis=1,keepdims=True)
global_nearest = ak.flatten(nearest_ind) + 1 # 1 = RR, 2 = LR, 3 = LL
global_nearest = ak.fill_none(global_nearest,0) # 0 = no vertices in the event!
has_vtx = global_nearest > 0
nearest_vtx = ak.flatten(all_vtx_nearest[nearest_ind])
dist_nearest = ak.flatten(all_dist_nearest[nearest_ind])

dist_rr_nearest = ak.flatten(dist_rr_nearest)
dist_lr_nearest = ak.flatten(dist_lr_nearest)
dist_ll_nearest = ak.flatten(dist_ll_nearest)

has_rr = ak.count(vtx_regreg.x,axis=1) > 0
has_lr = ak.count(vtx_lowreg.x,axis=1) > 0
has_ll = ak.count(vtx_lowlow.x,axis=1) > 0

In [14]:
nearest_vtx.reduced_chi2[has_vtx]

<Array [11.7, -1e+04, ... 0.0104, 571] type='13210 * ?float32'>